In [1]:

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import random

In [2]:
# Function to get error code based on the error message
def get_error_code(error_message):
    if 'billed customer' in error_message:
        return 1
    elif 'Phone number not valid' in error_message:
        return 3
    elif 'no longer in service' in error_message:
        return 2
    elif '' in error_message:
        return 0
    else:
        return -1  # Unknown error message

In [6]:
# Read phone numbers from file and remove country code
with open('USCCphonenumber.txt', 'r') as file:
    phone_numbers = [line.strip()[1:] if line.strip().startswith('1') else line.strip() for line in file]

In [4]:
options = webdriver.ChromeOptions()
options.headless = True

results = []
consecutive_error_count = 0
current_index = 0  # Initialize the index to keep track of the current phone number

def init_driver():
    """Initialize the Chrome WebDriver."""
    driver = webdriver.Chrome(options=options)
    driver.get('https://prepaid.uscellular.com/refill/dashboard')
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))
    ).click()
    return driver

# Initialize the WebDriver for the first time
driver = init_driver()

while current_index < len(phone_numbers):
    print(current_index)
    # try:
    phone_number = phone_numbers[current_index]
    print(f"Processing phone number {phone_number} at index {current_index}")

    # phone number input field
    phone_input_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "phone"))
    )
    phone_input_field.click()  # Focus on the input field

    # Clear the input field 
    phone_input_field.send_keys(Keys.CONTROL + "a")
    phone_input_field.send_keys(Keys.DELETE)
        
    # Clear the input field and input phone number
    # phone_input_field.clear()
    for char in phone_number:
        phone_input_field.send_keys(char)
        time.sleep(random.uniform(0.3, 1))  # Small delay between each character

    # default error message
    error_message = ""
    error_code = get_error_code(error_message)  # Get error code

    # Check for an error message, wait up to 5 seconds
    try:
        error_message_xpath = "/html/body/div[1]/main/div[1]/div/div[2]/div/div[2]/div[1]/div/div[2]/div/div/div/div/form/div[1]/div[1]/div[2]"
        error_message_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, error_message_xpath))
        )
        if error_message_element.text.strip():  # If there's text in the error element
            error_message = error_message_element.text
            print(error_message)
            error_code = get_error_code(error_message)  # Get error code
            print(error_code)
    except TimeoutException:
        print("No error message appeared within 5 seconds")
        error_code == -2 # If no error message appeared within 5 seconds, assume no error
        # pass

        # Reset consecutive error count if no error occurred
    if error_code == 0:
        print('Error code is: ',error_code)
        consecutive_error_count = 0
        results.append((phone_number, error_code))
        current_index += 1
            
    elif error_code == -2:
        print('Error code is: ',error_code)
        driver.quit()
        time.sleep(random.uniform(5, 20))
        driver = init_driver()
        consecutive_error_count = 0
        print("Resetting WebDriver")
        continue

    else:
        print('haha')
        print('Error code is: ',error_code)
        consecutive_error_count += 1
        results.append((phone_number, error_code))
        current_index += 1
        if consecutive_error_count == 2:  # If three consecutive errors, reset
            driver.quit()
            time.sleep(random.uniform(5, 20))
            driver = init_driver()
            consecutive_error_count = 0
            print("Resetting WebDriver")


    # except Exception as e:
    #     print(f"An error occurred: {e}. Retrying the current phone number.")
    #     driver.quit()
    #     time.sleep(random.uniform(20, 30))
    #     print(current_index)
    #     driver = init_driver()  # Re-initialize the WebDriver

# Clean up
driver.quit()


0
Processing phone number 2073571429 at index 0
No error message appeared within 5 seconds
Error code is:  0
1
Processing phone number 4237216670 at index 1


ElementClickInterceptedException: Message: element click intercepted: Element <input aria-invalid="false" name="phone" placeholder=" " type="text" inputmode="numeric" aria-label="UScellular Prepaid Number" class="MuiInputBase-input MuiOutlinedInput-input MuiInputBase-inputAdornedEnd MuiOutlinedInput-inputAdornedEnd" value="(207) 357-1429"> is not clickable at point (504, 375). Other element would receive the click: <iframe title="recaptcha challenge expires in two minutes" name="c-azicl9zefu57" frameborder="0" scrolling="no" sandbox="allow-forms allow-popups allow-same-origin allow-scripts allow-top-navigation allow-modals allow-popups-to-escape-sandbox" src="https://www.google.com/recaptcha/api2/bframe?hl=en&amp;v=3sU2vDRVDmUU2E0Ro4VadvPr&amp;k=6LeuwcIUAAAAAOxIwk4Ji5ZrvseZel4_ufSQ7Njg" style="width: 400px; height: 580px;"></iframe>
  (Session info: chrome=118.0.5993.120)
Stacktrace:
	GetHandleVerifier [0x00007FF67E158EF2+54786]
	(No symbol) [0x00007FF67E0C5612]
	(No symbol) [0x00007FF

In [5]:

options = webdriver.ChromeOptions()
options.headless = True

results = []
consecutive_error_count = 0
current_index = 0  # Initialize the index to keep track of the current phone number

def init_driver():
    """Initialize the Chrome WebDriver."""
    # ... [The rest of the init_driver function]

# Initialize the WebDriver for the first time
driver = init_driver()

while current_index < len(phone_numbers):
    print(current_index)
    phone_number = phone_numbers[current_index]
    print(f"Processing phone number {phone_number} at index {current_index}")

    # phone number input field
    phone_input_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "phone"))
    )
    phone_input_field.click()  # Focus on the input field

    # Clear the input field 
    phone_input_field.send_keys(Keys.CONTROL + "a")
    phone_input_field.send_keys(Keys.DELETE)
    
    # Clear the input field and input phone number
    # phone_input_field.clear()
    for char in phone_number:
        phone_input_field.send_keys(char)
        time.sleep(random.uniform(0.3, 1))  # Small delay between each character

    # default error message
    error_message = ""
    error_code = get_error_code(error_message)  # Get error code

    # Check for an error message, wait up to 5 seconds
    try:
        error_message_xpath = "/html/body/div[1]/main/div[1]/div/div[2]/div/div[2]/div[1]/div/div[2]/div/div/div/div/form/div[1]/div[1]/div[2]"
        error_message_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, error_message_xpath))
        )
        if error_message_element.text.strip():  # If there's text in the error element
            error_message = error_message_element.text
            print(error_message)
            error_code = get_error_code(error_message)  # Get error code
            print(error_code)
    except TimeoutException:
        print("No error message appeared within 5 seconds")
        error_code = -2 # If no error message appeared within 5 seconds, assume no error

    # Reset consecutive error count if no error occurred
    if error_code == 0:
        print('Error code is: ', error_code)
        consecutive_error_count = 0
        results.append((phone_number, error_code))
        current_index += 1
        
    elif error_code == -2:
        print('Error code is: ', error_code)
        driver.quit()
        time.sleep(random.uniform(5, 20))
        driver = init_driver()
        consecutive_error_count = 0
        print("Resetting WebDriver")
        # continue is not necessary here because it's the last statement in the loop

    else:
        print('Error code is: ', error_code)
        consecutive_error_count += 1
        results.append((phone_number, error_code))
        if consecutive_error_count == 2:  # If three consecutive errors, reset
            driver.quit()
            time.sleep(random.uniform(5, 20))
            driver = init_driver()
            consecutive_error_count = 0
            print("Resetting WebDriver")
        # Always increment the current_index even if there's an error
        current_index += 1

# Clean up
driver.quit()


0
Processing phone number 2073571429 at index 0


AttributeError: 'NoneType' object has no attribute 'find_element'

In [12]:
df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code'])
df_results

,Phone Number,Error Code
0,2073571429,0
1,4237216670,0
2,3149108829,0
3,2172487730,0
4,9193308497,0
5,5732416696,0
6,3607512985,0
7,7153401533,0
8,6414852424,0
9,8036592837,0


In [8]:
# Convert the results to a DataFrame 
df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code'])
print(df_results)

  Phone Number  Error Code
0   2073571429           0
1   4237216670           3
2   3149108829           3
3   2172487730           2
4   9193308497           1
5   5732416696           3
6   3607512985           1
7   7153401533           3
8   6414852424           3
9   8036592837           3


In [9]:
import os

csv_file_path = 'uscs_query_result.csv'

df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code'])
if os.path.exists(csv_file_path):
    df_existing = pd.read_csv(csv_file_path)
    
    last_index = df_existing.index[-1] if not df_existing.empty else -1

    df_results.index = range(last_index + 1, last_index + 1 + len(df_results))
    
    # Append the new results to the existing DataFrame
    df_final = df_existing.append(df_results)
    df_final = df_final.iloc[last_index + 1:,1:]
else:
    df_final = df_results
    df_final.index = range(len(df_final))

# Output to CSV, without overwriting existing data and with continuous indexing
df_final.to_csv(csv_file_path, index_label='Index', mode='w' if not os.path.exists(csv_file_path) else 'a', header=not os.path.exists(csv_file_path))

print(f"Results have been appended to {csv_file_path}")


Results have been appended to uscs_query_result.csv


C:\Users\ericluo\AppData\Local\Temp\ipykernel_13412\1051452944.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_existing.append(df_results)
